# SQL-запросы

Проанализируем базу данных в сервисе для чтения книг по подписке.

В базе данных есть информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

Описание данных:
- Таблица `books` (Содержит данные о книгах):
    - `book_id` — идентификатор книги;
    - `author_id` — идентификатор автора;
    - `title` — название книги;
    - `num_pages` — количество страниц;
    - `publication_date` — дата публикации книги;
    - `publisher_id` — идентификатор издателя.
- Таблица `authors` (Содержит данные об авторах):
    - `author_id` — идентификатор автора;
    - `author` — имя автора.
- Таблица `publishers` (Содержит данные об издательствах):
    - `publisher_id` — идентификатор издательства;
    - `publisher` — название издательства;
- Таблица `ratings` (Содержит данные о пользовательских оценках книг):
    - `rating_id` — идентификатор оценки;
    - `book_id` — идентификатор книги;
    - `username` — имя пользователя, оставившего оценку;
    - `rating` — оценка книги.
- Таблица `reviews` (Содержит данные о пользовательских обзорах):
    - `review_id` — идентификатор обзора;
    - `book_id` — идентификатор книги;
    - `username` — имя автора обзора;
    - `text` — текст обзора.

## Подключение к базе данных

Загрузим необходимые библиотеки.

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

Подключимся к базе данных.

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследование таблиц

Для начала созданим функцию для выполнения и отоброжения запросов.

In [3]:
def show_sql(query):
    con=engine.connect()      
    return pd.io.sql.read_sql(sql=text(query), con = con)

Теперь познакомимся со всеми таблицами в базе данных.

Выведем первые 5 строк из таблицы books.

In [4]:
query = '''
SELECT *
FROM books
LIMIT 5
'''

show_sql(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Выведем первые 5 строк из таблицы authors.

In [5]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''

show_sql(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Выведем первые 5 строк из таблицы publishers.

In [6]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''

show_sql(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Выведем первые 5 строк из таблицы ratings.

In [7]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''

show_sql(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Выведем первые 5 строк из таблицы reviews.

In [8]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''

show_sql(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


С таблицами познакомились, переходим к запросам.

## Запросы

Посчитаем, сколько книг вышло после 1 января 2000 года.

In [9]:
query = '''
SELECT COUNT(book_id) AS total
FROM books
WHERE publication_date > DATE('2000-01-01')
LIMIT 5
'''

show_sql(query)

,total
0,819


После 1 января 2000 года вышло 819 книг.

Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [10]:
query = '''
SELECT b.title,
    ta.total_reviews,
    ta.average_rating
FROM (
    SELECT rat.book_id AS book_id,
        COUNT(DISTINCT rev.review_id) AS total_reviews,
        ROUND(AVG(rat.rating), 2) AS average_rating
    FROM ratings AS rat
    FULL JOIN reviews AS rev ON rat.book_id = rev.book_id
    GROUP BY rat.book_id) AS ta
JOIN books AS b ON b.book_id = ta.book_id
'''

show_sql(query)

,title,total_reviews,average_rating
0,'Salem's Lot,2,3.67
1,1 000 Places to See Before You Die,1,2.50
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.67
3,1491: New Revelations of the Americas Before C...,2,4.50
4,1776,4,4.00
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.67
996,Xenocide (Ender's Saga #3),3,3.40
997,Year of Wonders,4,3.20
998,You Suck (A Love Story #2),2,4.50


Посчитали количество обзоров и среднюю оценку для каждой книги.

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры.

In [11]:
query = '''
SELECT p.publisher,
    tb.total_books
FROM (
    SELECT publisher_id,
        COUNT(book_id) AS total_books
    FROM books
    WHERE num_pages > 50
    GROUP BY publisher_id
    ORDER BY total_books DESC
    LIMIT 1) AS tb
JOIN publishers AS p ON tb.publisher_id = p.publisher_id
'''

show_sql(query)

,publisher,total_books
0,Penguin Books,42


Издательство Penguin Books выпустило 42 книги толще 50 страниц.

Определим автора с самой высокой средней оценкой книг — будем учитывать только книги с 50 и более оценками.

In [12]:
query = '''
SELECT a.author,
    ROUND(ar.average_rating, 2) AS average_rating
FROM (
    SELECT author_id,
        AVG(r.rating) AS average_rating
    FROM books AS b
    JOIN ratings AS r ON b.book_id = r.book_id
    WHERE b.book_id in (
        SELECT book_id
        FROM ratings
        GROUP BY book_id
        HAVING COUNT(rating_id) >= 50)
    GROUP BY author_id
    ORDER BY average_rating DESC
    LIMIT 1) AS ar
JOIN authors AS a ON a.author_id = ar.author_id
'''

show_sql(query)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.29


У автора J.K. Rowling/Mary GrandPré самая высокая средняя оценка книг с учетом книг только с 50 и более оценками - 4.29

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [13]:
query = '''
SELECT AVG(total_reviews) AS average_reviews
FROM (
    SELECT rat.username,
        COUNT(DISTINCT review_id) AS total_reviews
    FROM ratings AS rat
    JOIN reviews AS rev ON rat.username = rev.username
    GROUP BY rat.username
    HAVING COUNT(DISTINCT rat.rating_id) > 48) AS tr
'''

show_sql(query)

,average_reviews
0,24.0


В среднем 24 обзора от пользователей, которые поставили больше 48 оценок.

## Вывод

- После 1 января 2000 года вышло 819 книг;
- Посчитали количество обзоров и среднюю оценку для каждой книги;
- Издательство Penguin Books выпустило 42 книги толще 50 страниц;
- У автора J.K. Rowling/Mary GrandPré самая высокая средняя оценка книг с учетом книг только с 50 и более оценками - 4.29;
- В среднем 24 обзора от пользователей, которые поставили больше 48 оценок.